In [1]:
import numpy as np
from collections import Counter

# Paths
X_PATH = r"C:\Users\user\Desktop\GaitLab\GaitLab\src\keypoints_csv\ts_data\X_seq_len64_stride32.npy"
y_PATH = r"C:\Users\user\Desktop\GaitLab\GaitLab\src\keypoints_csv\ts_data\y_seq_len64_stride32.npy"

# Load sequences and labels
X = np.load(X_PATH, allow_pickle=True)
y = np.load(y_PATH, allow_pickle=True)

# Basic checks
print("Loaded X shape:", X.shape)
print("Loaded y shape:", y.shape)
print("NaNs in X:", np.isnan(X).sum())
print("Infs in X:", np.isinf(X).sum())

# Label distribution
print("\nLabel distribution (counts):")
print(Counter(y))

# Unique classes
class_names = np.unique(y)
print("\nUnique labels (classes):", class_names)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\GaitLab\\GaitLab\\src\\keypoints_csv\\ts_data\\X_seq_len64_stride32.npy'

In [ ]:
from

NameError: name 'X' is not defined

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_teacher_model(input_shape, num_classes):
    """
    High-capacity TCN teacher model.
    """
    inputs = layers.Input(shape=input_shape, name="input_layer")
    x = inputs
    
    # --- TCN Blocks (6 blocks, wider filters) ---
    for i in range(6):
        x_prev = x
        x = layers.Conv1D(filters=128, kernel_size=3, padding='causal', activation=None, name=f'conv1d_teacher_{i}')(x)
        x = layers.BatchNormalization(name=f'bn_teacher_{i}')(x)
        x = layers.Activation('relu', name=f'act_teacher_{i}')(x)
        x = layers.Dropout(0.2, name=f'drop_teacher_{i}')(x)
        # Residual connection
        if x_prev.shape[-1] == x.shape[-1]:
            x = layers.Add()([x_prev, x])
    
    # Global pooling
    x = layers.GlobalAveragePooling1D(name="gap_teacher")(x)
    
    # Feed-forward for extra capacity
    x = layers.Dense(256, activation='relu', name="ff_teacher")(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax', name="output_teacher")(x)
    
    model = models.Model(inputs=inputs, outputs=outputs, name="teacher_model")
    return model

# --- Build & compile ---
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = len(np.unique(y_train))

teacher = build_teacher_model(input_shape, num_classes)
teacher.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

teacher.summary()


NameError: name 'X_train' is not defined

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# --- Encode class labels to integers ---
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)
y_test_enc = le.transform(y_test)

# Optional: save class names
class_names = le.classes_
print("Class names:", class_names)


In [4]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# --- Callbacks ---
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_teacher_model.keras', monitor='val_loss', save_best_only=True)

# --- Train teacher model ---
history_teacher = teacher.fit(
    X_train, y_train_enc,
    validation_data=(X_val, y_val_enc),
    epochs=50,           # increase later if needed
    batch_size=16,       # CPU-friendly
    callbacks=[early_stop, checkpoint],
    verbose=2
)

# --- Evaluate on test set ---
test_loss, test_acc = teacher.evaluate(X_test, y_test_enc, verbose=0)
print("Teacher Test Accuracy: {:.2f}%".format(test_acc * 100))

# --- Predictions & classification report ---
y_pred = np.argmax(teacher.predict(X_test), axis=1)
print(classification_report(y_test_enc, y_pred, target_names=class_names))


NameError: name 'teacher' is not defined